In [1]:
import pyspark as ps
from pyspark.ml import Pipeline
from pyspark.mllib.recommendation import ALS as lib_ALS
from pyspark.mllib.evaluation import RegressionMetrics
import math

In [11]:
spark = ps.sql.SparkSession.builder \
            .master("local[4]") \
            .appName("building recommender") \
            .getOrCreate()
            
sc = spark.sparkContext

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:57831)
Traceback (most recent call last):
  File "/Users/davidclausen/spark-2.1.0-bin-hadoop2.7/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 963, in start
    self.socket.connect((self.address, self.port))
  File "/Users/davidclausen/anaconda2/lib/python2.7/socket.py", line 228, in meth
    return getattr(self._sock,name)(*args)
error: [Errno 61] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:57831)

In [3]:
subset_raw_data = sc.textFile('data/subset.csv')

In [4]:
print type(subset_raw_data)
subset_raw_data.take(3)

<class 'pyspark.rdd.RDD'>


[u',id,source,reader_id,from_book_id,book_id,ad_id,boost_id,clicked,claimed,optin,created_at,updated_at',
 u'29295882,29295919,success_page,186643,20370,19203,0,0,1,1,0,2017-02-20 02:16:11,2017-02-20 02:26:24',
 u'29295883,29295920,success_page,186643,20370,19813,0,0,0,0,0,2017-02-20 02:16:11,2017-02-20 02:16:11']

In [5]:
subset_raw_data_header = subset_raw_data.take(1)[0]
subset_raw_data_header

u',id,source,reader_id,from_book_id,book_id,ad_id,boost_id,clicked,claimed,optin,created_at,updated_at'

In [6]:
subset_data = subset_raw_data.filter(lambda line: line!=subset_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (tokens[3],tokens[5],tokens[9])).cache()

In [7]:
subset_data.take(4)

[(u'186643', u'19203', u'1'),
 (u'186643', u'19813', u'0'),
 (u'523754', u'16281', u'0'),
 (u'523754', u'22669', u'1')]

In [8]:
training_RDD, validation_RDD, test_RDD = subset_data.randomSplit([6, 2, 2], seed=0L)
validation_for_predict_RDD = validation_RDD.map(lambda x: (x[0], x[1]))
test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))

In [9]:
type(training_RDD)

pyspark.rdd.PipelinedRDD

In [10]:
seed = 5L
iterations = 20
regularization_parameter = 0.05
ranks = [10, 15, 20, 25, 30]
errors = [0, 0, 0, 0, 0]
reg_met = [0, 0, 0, 0, 0]
err = 0
tolerance = 0.02

min_error = float('inf')
best_rank = -1
best_iteration = -1
for rank in ranks:
    model = lib_ALS.train(training_RDD, rank=rank, seed=seed, iterations=iterations,\
                      lambda_=regularization_parameter)
#     model = lib_ALS.trainImplicit(training_RDD, rank=rank, seed=seed, iterations=iterations,\
#                               lambda_=regularization_parameter)
    predictions = model.predictAll(validation_for_predict_RDD)\
                        .map(lambda r: ((r[0], r[1]), r[2]))

    rates_and_preds = validation_RDD\
                        .map(lambda r: ((int(r[0]), int(r[1])), float(r[2])))\
                        .join(predictions)
    valuesAndPreds = rates_and_preds\
                        .map(lambda p: (p[1]))
    metrics = RegressionMetrics(valuesAndPreds)
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    errors[err] = error
    reg_met[err] = metrics.rootMeanSquaredError
    err += 1
    print 'For rank %s the RMSE is %s the reg_met is %s' % (rank, error,\
                                                            metrics.rootMeanSquaredError)
    if error < min_error:
        min_error = error
        best_rank = rank

print 'The best model was trained with rank %s' % best_rank

Traceback (most recent call last):
  File "/Users/davidclausen/anaconda2/lib/python2.7/SocketServer.py", line 290, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/davidclausen/anaconda2/lib/python2.7/SocketServer.py", line 318, in process_request
    self.finish_request(request, client_address)
  File "/Users/davidclausen/anaconda2/lib/python2.7/SocketServer.py", line 331, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/davidclausen/anaconda2/lib/python2.7/SocketServer.py", line 652, in __init__
    self.handle()
  File "/Users/davidclausen/spark-2.1.0-bin-hadoop2.7/python/pyspark/accumulators.py", line 235, in handle
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/Users/davidclausen/spark-2.1.0-bin-hadoop2.7/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 883, in send_command
    response = connection.send_command(command)
  File "/Users/davidclau

----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 57843)


    num_updates = read_int(self.rfile)
  File "/Users/davidclausen/spark-2.1.0-bin-hadoop2.7/python/pyspark/serializers.py", line 557, in read_int
    raise EOFError
EOFError
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:57831)
Traceback (most recent call last):
  File "/Users/davidclausen/spark-2.1.0-bin-hadoop2.7/python/lib/py4j-0.10.4-src.zip/py4j/java_gateway.py", line 963, in start
    self.socket.connect((self.address, self.port))
  File "/Users/davidclausen/anaconda2/lib/python2.7/socket.py", line 228, in meth
    return getattr(self._sock,name)(*args)
error: [Errno 61] Connection refused


----------------------------------------


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:57831)

In [ ]:
type(subset_data.take(1)[0][0])

In [ ]:
rates_and_preds.take(5)

In [ ]:
predictions.take(5)

In [ ]:
valuesAndPreds.take(5)